In [39]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import xarray as xr
import intake,yaml
import intake_esm
import numpy as np
#anonymous True access to test data s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Amon/tas/gr1/v20190726
##importing packages needed

In [40]:
col_url = "gfdltest.json" 

In [41]:
col = intake.open_esm_datastore(col_url)

In [42]:
col.df

product_id  institute      model  experiment frequency modeling_realm  \
0     output  NOAA-GFDL  GFDL-ESM4  historical       mon          atmos   
1     output  NOAA-GFDL  GFDL-ESM4  historical       mon          atmos   
2     output  NOAA-GFDL  GFDL-ESM4      ssp370       mon          atmos   

  mip_table ensemble_member variable temporal_subset    version  \
0      Amon        r1i1p1f1      tas   185001-194912  v20190726   
1      Amon        r1i1p1f1      tas   195001-201412  v20190726   
2      Amon        r1i1p1f1      tas   201501-210012  v20190726   

                                                path  
0  s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...  
1  s3://gfdl-esgf/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...  
2  s3://gfdl-esgf/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...

In [43]:
dset_dict = col.to_dataset_dict(cdf_kwargs={'chunks': {'time': 20}}, storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'product_id.institute.model.modeling_realm.experiment.frequency.mip_table'


In [16]:
#Alternate way to just search for what we want expname_filter = ['historical']
#table_id_filter = 'Amon'
#model_filter = 'GFDL-ESM4'
#variable_id_filter = "tas"
#cat = col.search(experiment=expname_filter, mip_table=table_id_filter,model=model_filter)

In [51]:
hxr_gfdl_esm4= dset_dict["output.NOAA-GFDL.GFDL-ESM4.atmos.historical.mon.Amon"]
fxr_gfdl_esm4= dset_dict["output.NOAA-GFDL.GFDL-ESM4.atmos.ssp370.mon.Amon"]

#fxr = xr.open_dataset(file_future)
#hxr = xr.open_dataset(file_hist)

! GLOBAL MEAN 
let hgtas = tas[x=@ave,y=@ave,d=1,l=529:780@ave]


In [45]:
lat1 = 60
lat2 = 90

In [46]:
tas = hxr.tas
weights = np.cos(np.deg2rad(tas.lat))
weights.name = "weights"

!!!!!!!!!!!!!!!! COMPUTE BASELINE (1994-2014) TAS !!!!!!!!!!!!!!!!!!!!!!!!

In [52]:
hgtas = hxr_gfdl_esm4.tas.sel(time=slice("1994", "2014"))
hgtas = hgtas.weighted(weights)
hgtas = hgtas.mean(dim=['lon','lat','time'])
hgtas = hgtas.values
print(hgtas) #(287.2) from ferret

[287.16173045]


In [53]:
hatas = hxr_gfdl_esm4.tas.sel(time=slice("1994", "2014"),lat=slice(lat1, lat2))
hatas = hatas.weighted(weights)
hatas = hatas.mean(dim=['lon','lat','time'])
hatas = hatas.values 
print(hatas)#hatas 264.5 from Ferret

[264.3833672]


In [49]:
print(hgtas,hatas) #hatas 264.5 from Ferret

[287.16173045] [264.3833672]


!!!!!!!!!!!!!!! COMPUTE SHORT-TERM (2020-2040) TAS !!!!!!!!!!!!!!!!!!!

In [54]:
fgtas = fxr_gfdl_esm4.tas.sel(time=slice("2020", "2040"))
fgtas = fgtas.weighted(weights)
fgtas = fgtas.mean(dim=['lon','lat','time'])
fgtas_st = fgtas.values
print(fgtas_st) #287.8 from Ferret

[287.75333068]


In [55]:
fatas = fxr_gfdl_esm4.tas.sel(time=slice("2020", "2040"),lat=slice(lat1, lat2))
fatas = fatas.weighted(weights)
fatas = fatas.mean(dim=['lon','lat','time'])
fatas_st = fatas.values
print(fatas_st) #265.6 From ferret

[265.49362442]


!!!!!!!!!!!!!!!! COMPUTE LONG-TERM (2080-2100) TAS !!!!!!!!!!!!!!!!!!!

In [56]:
fgtas_lt = fxr_gfdl_esm4.tas.sel(time=slice("2080", "2100"))
fgtas_lt = fgtas_lt.weighted(weights)
fgtas_lt = fgtas_lt.mean(dim=['lon','lat','time'])
fgtas_lt = fgtas_lt.values
print(fgtas_lt) #289.8 from Ferret

[289.80264889]


In [57]:
fatas_lt = fxr_gfdl_esm4.tas.sel(time=slice("2080", "2100"),lat=slice(lat1, lat2))
fatas_lt = fatas_lt.weighted(weights)
fatas_lt = fatas_lt.mean(dim=['lon','lat','time'])
fatas_lt = fatas_lt.values
print(fatas_lt) #269.8 from Ferret

[269.77492881]


!!!!!!!!!!!!!!!!!!!! COMPUTE SHORT-TERM POLAR AMP !!!!!!!!!!!!!!!!!!!!

GLOBAL WARMING

In [58]:
g_tdiff_st = fgtas_st - hgtas
g_tdiff_st

array([0.59160023])

ARCTIC WARMING

In [59]:
a_tdiff_st = fatas_st - hatas
a_tdiff_st

array([1.11025722])

! POLAR AMPLIFICATION SHORT-TERM (1.1/0.6=1.83)

In [60]:
pamp_st = a_tdiff_st / g_tdiff_st
pamp_st

array([1.87670178])

!!!!!!!!!!!!!!!!!!!! COMPUTE LONG-TERM POLAR AMP !!!!!!!!!!!!!!!!!!!!!

In [61]:
#GLOBAL WARMING
print(fgtas_lt,hgtas)
g_tdiff_lt = fgtas_lt - hgtas
g_tdiff_lt #2.64

[289.80264889] [287.16173045]


array([2.64091844])

In [62]:
#ARCTIC WARMING
a_tdiff_lt = fatas_lt - hatas
a_tdiff_lt #5.391561607460574


array([5.39156161])

! POLAR AMPLIFICATION LONG-TERM (5.3/2.6=2.04)

In [63]:
pamp_lt = a_tdiff_lt / g_tdiff_lt
pamp_lt #2.0415479396985075

array([2.04154794])

In [64]:
print("POLAR AMP SHORT TERM:", pamp_st) #1.8 from Ferret
print("POLAR AMP LONG TERM:", pamp_lt) #2.0 from Ferret

POLAR AMP SHORT TERM: [1.87670178]
POLAR AMP LONG TERM: [2.04154794]
